In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
import folium
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [2]:
df=pd.read_csv('merged.csv')

In [3]:
pd.set_option('display.max_columns', None)


In [4]:
df=pd.DataFrame(df)
df.head(20)
drop= df['inspector'].value_counts() < 20

In [5]:
df= df.drop(df[['transactions', 'price', 'establishment_x', 'establishment_y', 'address_x', 'address_y', 'trimmed_name_x', 'trimmed_name_y', 'clean_address_x', 'clean_address_y', 'address_distance', 'name_distance']], axis=1)

In [6]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 
df.describe()

,Unnamed: 0,number_of_violations,zip_code,house_numbers,distance,review_count,longitude,latitude,sector,total_score,rating
count,1820.000000,1820.000000,1820.000000,1820.00000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000,1820.000000
mean,1166.184066,5.082967,78220.814286,4784.15989,7830.619272,210.672527,-98.514171,29.489581,15.985714,93.996154,3.678571
std,709.624182,4.892942,15.141757,4756.65303,3071.065979,288.162365,0.058245,0.055954,9.218967,5.436068,0.669349
min,0.000000,0.000000,78201.000000,21.00000,811.199667,22.000000,-98.708442,29.354318,1.000000,56.000000,1.000000
25%,529.500000,2.000000,78209.000000,821.00000,5929.771588,55.000000,-98.564447,29.433901,8.000000,91.000000,3.500000
50%,1171.500000,4.000000,78216.000000,2918.00000,7986.163583,109.000000,-98.495517,29.492450,16.000000,95.000000,4.000000
75%,1782.250000,7.000000,78230.000000,8226.50000,9377.441334,251.000000,-98.480431,29.530199,22.000000,98.000000,4.000000
max,2408.000000,50.000000,78259.000000,22211.00000,20222.717722,2661.000000,-98.398119,29.661520,35.000000,100.000000,5.000000


In [7]:
#Dropped all establishments with less than 22 reviews
df=  df[df['review_count'] >= 22] 

In [8]:
#grouped health scores into 5 groups with a range of three
health_scores= {'health_scores' : []}
for x in df['total_score']:
#     print(x)
    if x > 95:
        health_scores['health_scores'].append('1')
    elif x> 90:
        health_scores['health_scores'].append('2')
    else:
        health_scores['health_scores'].append('3')       
#df
df['health_scores']= health_scores['health_scores']  


In [9]:
#encode for the inspectors
df['inspector'].value_counts()

Lauren Schumacher     180
Jeffrey Carrizales    136
RYAN DAVIS            129
Jacob Johanson        128
Sumner Bumgardner     117
Bamba Njie             89
Diana Garcia           88
Deborah Liu            82
Bernard Capdeboscq     79
Cicily Martinez        77
Valerie Suarez         75
Benito Zuniga          72
Daisy Zamora           68
Nathan Balfour         64
Raul Jimenez           38
John Payelle           38
William Hudson         37
Kassandra Aguilar      35
Debra Hernandez        31
Erica Arreola          27
Mario Cavazos          27
MICHELLE MORENO        26
William Emminger       26
Kathleen Prenzler      26
Amanda Cantu           22
Kacie Patrick          20
Adan Zamora            18
Sean Gilbert           16
Fermin Garza           15
Victoria Paredes        7
Monica Caballero        7
Juanicia Page           6
Joe Mokry               5
Daniel Guzman           3
Maurico Ripley          3
Erica Llanas            2
Rebecca Vera            1
Name: inspector, dtype: int64

In [10]:
#Dropping inspectors that have not copmpleted AT LEAST 20 inspections
vdf = pd.DataFrame(df['inspector'].value_counts())
vdf['check'] = vdf['inspector'] > 1
over_20 = list(vdf.loc[vdf['check'], :].reset_index()['index'])

In [11]:
inspectors = pd.DataFrame(df.inspector.unique())

In [12]:
df['check'] = df['inspector'].isin(over_20)
new_df = df.loc[df['check'], :]
new_df = new_df.drop(columns = ['check', 'Unnamed: 0'])
new_df.head()

,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores
0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1
1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,1
3,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,2
4,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,1
5,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5,1


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [14]:
#Will now encode the inspector column to use that as an input for the model

In [15]:
#Label Encoding the inspector column
label_enc= LabelEncoder()
new_df['inspector_encode'] = label_enc.fit_transform(new_df.inspector)

In [16]:
#OneHot Encoding
one_enc = OneHotEncoder()
X = one_enc.fit_transform(new_df.inspector_encode.values.reshape(-1,1)).toarray()
print(X)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [17]:
#Joining the encoded data to the main Df
dfOneHot = pd.DataFrame(X, columns = ["health_inspector_"+str(int(i)) for i in range(X.shape[1])])

In [18]:
new_df = new_df.reset_index()
new_df = new_df.reset_index()
new_df['index']= new_df['level_0']
new_df = new_df.drop(columns = ['level_0'])
dfOneHot = dfOneHot.reset_index()
enc_df=new_df.join(dfOneHot, on = 'index', lsuffix='_left', rsuffix='_right')

In [37]:
new_df.head()

,index,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores,inspector_encode
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1,5
1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,1,28
2,2,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,2,5
3,3,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,1,28
4,4,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5,1,28


In [36]:
enc_df.head()

,index_left,inspection_date,inspector,number_of_violations,violations,zip_code,house_numbers,categories,distance,review_count,longitude,latitude,name,sector,district,total_score,rating,health_scores,inspector_encode,index_right,health_inspector_0,health_inspector_1,health_inspector_2,health_inspector_3,health_inspector_4,health_inspector_5,health_inspector_6,health_inspector_7,health_inspector_8,health_inspector_9,health_inspector_10,health_inspector_11,health_inspector_12,health_inspector_13,health_inspector_14,health_inspector_15,health_inspector_16,health_inspector_17,health_inspector_18,health_inspector_19,health_inspector_20,health_inspector_21,health_inspector_22,health_inspector_23,health_inspector_24,health_inspector_25,health_inspector_26,health_inspector_27,health_inspector_28,health_inspector_29,health_inspector_30,health_inspector_31,health_inspector_32,health_inspector_33,health_inspector_34,health_inspector_35
0,0,2019-01-02,Cicily Martinez,3,"['22', '29', '45']",78212,2524,"['Italian', 'Desserts', 'Seafood']",4950.353662,99.0,-98.493780,29.453730,Capparelli's On Main,35,D,97,3.5,1,5,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2019-03-06,RYAN DAVIS,1,['31'],78230,14439,"['French', 'Asian Fusion', 'Tapas/Small Plates']",9312.955170,102.0,-98.544540,29.573330,Frederick's Bistro,9,A,98,4.0,1,28,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2019-11-04,Cicily Martinez,3,"['10', '15', '40']",78201,914,['Mexican'],3194.340357,42.0,-98.508619,29.466156,Fina's Kitchen,35,D,93,4.0,2,5,2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,2019-03-14,RYAN DAVIS,0,[],78230,13151,['Barbeque'],7935.733616,41.0,-98.537950,29.562850,Bill Miller Bar-B-Q,9,A,100,2.5,1,28,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2019-03-14,RYAN DAVIS,1,['21'],78230,10003,['American (Traditional)'],6996.407916,434.0,-98.536348,29.554391,Green Vegetarian Cuisine,9,A,98,4.5,1,28,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
#### Decision tree including number of review, total violations, yelp score, and inspector
#training / test split
X= enc_df.drop(columns=[ 'index_left', 'inspector_encode', 'index_right','inspection_date', 'inspector', 'violations', 'zip_code', 'house_numbers', 'categories', 'distance', 'longitude', 'latitude', 'name', 'sector', 'district', 'health_scores'])
y= enc_df['health_scores']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 123, stratify=enc_df.health_scores)
X_train.head()



,number_of_violations,review_count,total_score,rating,health_inspector_0,health_inspector_1,health_inspector_2,health_inspector_3,health_inspector_4,health_inspector_5,health_inspector_6,health_inspector_7,health_inspector_8,health_inspector_9,health_inspector_10,health_inspector_11,health_inspector_12,health_inspector_13,health_inspector_14,health_inspector_15,health_inspector_16,health_inspector_17,health_inspector_18,health_inspector_19,health_inspector_20,health_inspector_21,health_inspector_22,health_inspector_23,health_inspector_24,health_inspector_25,health_inspector_26,health_inspector_27,health_inspector_28,health_inspector_29,health_inspector_30,health_inspector_31,health_inspector_32,health_inspector_33,health_inspector_34,health_inspector_35
163,2,418.0,96,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
892,0,212.0,95,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
842,8,97.0,94,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
81,0,157.0,100,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,6,862.0,92,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [99]:
y_test.head()

1717    1
249     1
162     2
1401    2
404     1
Name: health_scores, dtype: object

In [100]:
#Train Model

#Create the Decision Tree Object


clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state= 42)

In [101]:
#Fit the model to the training data

clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [102]:
#enc_df['inspector'].value_counts()

In [103]:
#Estimate health group

y_pred = clf.predict(X_train)
y_pred[1:20]

array(['2', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '1', '3',
       '1', '3', '3', '1', '1', '1'], dtype=object)

In [104]:
#Estimate the probability of a health group
y_pred_proba = clf.predict_proba(X_train)
y_pred_proba

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [105]:
confusion_matrix(y_train, y_pred)

array([[683,   0,   0],
       [  0, 446,   0],
       [  0,   0, 326]])

In [106]:
#Accuracy: number of correct predictions over the number of total instances that have been evaluated.
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))

Accuracy of Decision Tree classifier on training set: 1.00


In [107]:
#Test Model

#Compute the accuracy of the model when run on the test data

print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on test set: 1.00


In [108]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00       683
           2       1.00      1.00      1.00       446
           3       1.00      1.00      1.00       326

    accuracy                           1.00      1455
   macro avg       1.00      1.00      1.00      1455
weighted avg       1.00      1.00      1.00      1455



In [109]:
#### Testing the accuracy with chi square 

Accuracy of Decision Tree classifier on training set: 1.00
